In [29]:
import pandas as pd

# read in

In [30]:
df = pd.read_csv('../../2_Feature_Engineering/export/combined_polly_bretschneider_iwg.csv')
df.head()

,raw_text,severity,binary_label
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False
1,Bevor Sie mit Jud...,1.0,True
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True
4,: Es muss in #Deutschland eine politische Kra...,1.0,True


# clean

In [31]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [32]:
df['cleaned_text'] = df['raw_text'].apply(ci)
df.head()

,raw_text,severity,binary_label,cleaned_text
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False,ekelhaft klos kannst gummistiefel gehen stinkt...
1,Bevor Sie mit Jud...,1.0,True,bevor judenstern markiert worden juden ganze ...
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False,de amis freut heuschrecken europa klein wirtsc...
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True,krim heißer tip investieren muß bürger landes...
4,: Es muss in #Deutschland eine politische Kra...,1.0,True,deutschland politische kraft geben sozial sch...


In [33]:
df.shape

(9169, 4)

# vectorize

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
vec = TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=100000)

In [36]:
X_features = vec.fit_transform(df['cleaned_text'])

In [37]:
X_features.shape

(9169, 100000)

# initialize model

In [38]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='liblinear', penalty='l2', C=0.23357214690901212) # params from earlier grid search

# Cross validation

In [39]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X_features, df['binary_label'], cv=5,
                        scoring=('f1', 'roc_auc', 'precision', 'recall'),
                        return_train_score=True)

In [40]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_f1',
 'test_precision',
 'test_recall',
 'test_roc_auc',
 'train_f1',
 'train_precision',
 'train_recall',
 'train_roc_auc']

In [41]:
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_f1,train_f1,test_roc_auc,train_roc_auc,test_precision,train_precision,test_recall,train_recall
0,0.030918,0.004987,0.776139,0.907467,0.925068,0.985410,0.921975,0.969408,0.670139,0.852967
1,0.031914,0.003989,0.769648,0.907860,0.926829,0.985864,0.928105,0.968812,0.657407,0.854124
2,0.028923,0.003989,0.736915,0.907775,0.918888,0.985460,0.909864,0.969737,0.619213,0.853256
3,0.032909,0.003989,0.754301,0.907270,0.930309,0.984929,0.930390,0.969707,0.634259,0.852388
4,0.033909,0.003989,0.768617,0.905695,0.929423,0.985301,0.901716,0.970559,0.669757,0.848958


In [42]:
# metrics
scores_df[['test_f1', 'test_roc_auc', 'test_precision', 'test_recall']].mean()

test_f1           0.761124
test_roc_auc      0.926103
test_precision    0.918410
test_recall       0.650155
dtype: float64